### Prime

###### Exercise 1: `mersenne_numbers`

A Mersenne number is any number that can be written as $2^p - 1$ for some $p$

In [3]:
def mersenne_number(p):
    return 2**p -1

mersenne_number(3)

7

Mersenne numbers can only be prime if their exponent, $p$, is prime. Make a list of the Mersenne numbers for all primes $p$ between 3 and 90 

In [4]:
def is_prime(num):
    if num<= 1:
        return False
    
    for fac in range(2, num):
        if num % fac ==0:
            return False
        
    return True

def print_primes(n):
    for num in range(1, n):
        if is_prime(num):
            print('%d is prime' % num)

In [5]:
print_primes(15)

2 is prime
3 is prime
5 is prime
7 is prime
11 is prime
13 is prime


In [6]:
def get_primes(start, end):
    return [num for num in range(start, end) if is_prime(num)]
    
#mersennes = [7] * 17
mersennes=[mersenne_number(p) for p in get_primes(3,  90)]
mersennes

[7,
 31,
 127,
 2047,
 8191,
 131071,
 524287,
 8388607,
 536870911,
 2147483647,
 137438953471,
 2199023255551,
 8796093022207,
 140737488355327,
 9007199254740991,
 576460752303423487,
 2305843009213693951,
 147573952589676412927,
 2361183241434822606847,
 9444732965739290427391,
 604462909807314587353087,
 9671406556917033397649407,
 618970019642690137449562111]

#####  `lucas_lehmer`

We can test if a Mersenne number is prime using the [Lucas-Lehmer test](https://en.wikipedia.org/wiki/Lucas%E2%80%93Lehmer_primality_test). First 

$$ n_0 = 4 $$
$$ n_i = (n_{i-1}^2 - 2) mod (2^p - 1) $$

Write a function that accepts the exponent $p$ of a Mersenne number and returns the Lucas-Lehmer sequence up to $i = p - 2$ (inclusive). 

In [7]:
def lucas_lehmer(p):
    lst=[]
    old=4
    lst.append(old)
    for n in range(p-2):
        new=(old**2-2)%mersenne_number(p)
        lst.append(new)
        old=new
    return lst
  
    
lucas_lehmer(17)

[4,
 14,
 194,
 37634,
 95799,
 119121,
 66179,
 53645,
 122218,
 126220,
 70490,
 69559,
 99585,
 78221,
 130559,
 0]

### `mersenne_primes`

For a given Mersenne number with exponent $p$, the number is prime if the Lucas-Lehmer series is 0 at position $p-2$. Write a function that tests if a Mersenne number with exponent $p$ is prime. Test if the Mersenne numbers with prime $p$ between 3 and 65 (i.e. 3, 5, 7, ..., 61) are prime. 
_HINT: The `zip` function is useful for combining two lists into a list of tuples_

In [8]:
def ll_prime(p):
    ll_sequence=lucas_lehmer(p)
    if ll_sequence[p-2]== 0:
        return 1
    else:
        return 0

In [9]:
mersenne_primes =[(p, ll_prime(p)) for p in range (3, 66) if is_prime(p)]
mersenne_primes

[(3, 1),
 (5, 1),
 (7, 1),
 (11, 0),
 (13, 1),
 (17, 1),
 (19, 1),
 (23, 0),
 (29, 0),
 (31, 1),
 (37, 0),
 (41, 0),
 (43, 0),
 (47, 0),
 (53, 0),
 (59, 0),
 (61, 1)]

### Optimize `is_prime`

Lets write the function to do this and check the speed!  `is_prime_fast` will take a number and return whether or not it is prime.

In [10]:
import math
def is_prime_fast(number):
    if number== 2:
        return True
    elif number <2:
        return False
    else:
        for factor in range (2, int(math.sqrt(number)+ 1)): ##inclusive
            if number%factor==0:
                return False
    return True
    

In [11]:
for n in range(10000):
    assert is_prime(n) == is_prime_fast(n)

Now lets check the timing, here we will use the `%%timeit` magic which will time the execution of a particular cell.

In [12]:
%%timeit
is_prime(67867967)

6.57 s ± 160 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
is_prime_fast(67867967)

2.14 ms ± 393 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now return a function which will find all prime numbers up to and including $n$.

In [15]:
def get_primes_fast(n):  # Using list comprehension 
    return [num for num in range(n) if is_prime_fast(num)]          # """Return a list of primes up to n

In [16]:
get_primes_fast(65)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61]

### sieve

In this problem we will develop an even faster method which is known as the Sieve of Eratosthenes (although it will be more expensive in terms of memory). The Sieve of Eratosthenes is an example of dynamic programming, where the general idea is to not redo computations we have already done (read more about it [here](https://en.wikipedia.org/wiki/Dynamic_programming)).

We have implemented a `sieve` function which will find all the prime numbers up to $n$. to implement the functions which it calls.  They are as follows

* `list_true` Make a list of true values of length $n+1$ where the first two values are false (this corresponds with step 1 of the algorithm above)
* `mark_false` takes a list of booleans and a number $p$.  Mark all elements $2p,3p,...n$ false (this corresponds with step 2 of the algorithm above)
* `find_next` Find the smallest `True` element in a list which is greater than some $p$ (has index greater than $p$ (this corresponds with step 3 of the algorithm above)
* `prime_from_list` Return indices of True values

In [19]:
def list_true(n):
    n_true=[True for i in range (n +1)]
    n_true[0]=False
    n_true[1]=False
    return n_true

In [20]:
list_true(6)

[False, False, True, True, True, True, True]

In [21]:
assert len(list_true(20)) == 21
assert list_true(20)[0] is False
assert list_true(20)[1] is False

we write a function which takes a list of elements and a number $p$ and marks elements false which are in the range $2p,3p ... N$.

In [22]:
def mark_false(bool_list, p):
    for i in range(2*p, len(bool_list), p):
        bool_list[i]=False
    return bool_list

In [24]:
assert mark_false(list_true(6), 2) == [False, False, True, True, False, True, False]

Now lets write a `find_next` function which returns the smallest element in a list which is not false and is greater than $p$.

In [25]:
def find_next(bool_list, p):
    for i, val in enumerate(bool_list):
        if val and i > p : # if val mean if val== true
            return i
    return None

In [26]:
find_next(list_true(6), 2)

3

In [27]:
assert find_next([True, True, True, True], 2) == 3
assert find_next([True, True, True, False], 2) is None

In [28]:
my_list=['a','b','c','d']
for i, val in enumerate(my_list):
    print (i, val)

0 a
1 b
2 c
3 d


Now given a list of `True` and `False`, return the index of the true values.

In [29]:
def prime_from_list(bool_list):
    return [i for i in range (len(bool_list)) if bool_list[i]]

In [30]:
assert prime_from_list([False, False, True, True, False]) ==  [2, 3]

In [31]:
def sieve(n):
    bool_list = list_true(n)
    p = 2
    while p is not None:
        bool_list = mark_false(bool_list, p)
        p = find_next(bool_list, p)
    return prime_from_list(bool_list)

In [32]:
assert sieve(1000) == get_primes(0, 1000)

In [33]:
%%timeit 
sieve(1000)

6.59 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
%%timeit 
get_primes(0, 1000)

20.1 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
